In [1]:
import findspark
findspark.init()
findspark.find()

'H:\\SPARK'

In [2]:
from pyspark.sql import SparkSession

# Initialize SparkSession with necessary configurations
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('Spark') \
    .config("spark.driver.memory", "15g") \
    .config("spark.hadoop.home.dir", "H:/HADOOP/") \
    .config("spark.hadoop.conf.dir", "H:/HADOOP/etc/hadoop/") \
    .getOrCreate()

import sys
sys.path.append("G:\Dissertation_Project")

# Get SparkContext from the SparkSession
sc = spark.sparkContext


KeyboardInterrupt: 

In [ ]:
spark

### BASE DATASET

In [ ]:
base_df = spark.read.csv("../../Data/Custom_Datasets/conversation_datasets_GPT.csv", header=True, inferSchema=True)
base_df.show(10, truncate=False)

+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------+
|Conversation_ID|Attacker_Helper                                                                                                                                                 |Victim                                                                                                                                                                                         |Conversation_Type|
+---------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------

### PREPROCESSED DATASET

In [ ]:
preprocessed_df = spark.read.csv("../../Data/Preprocessed_Datasets/GPT_dataset_preprocessed.csv", header=True, inferSchema=True)
preprocessed_df.show(10, truncate=False)

+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Convert Conversation Columns into actual Arrays

In [ ]:
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf
import ast

# UDF to convert string representation of list to actual list
def str_to_array_of_arrays(s):
    # Convert the string to a list and then wrap it inside another list
    return [ast.literal_eval(s)][0]

str_to_array_of_arrays_udf = udf(str_to_array_of_arrays, ArrayType(ArrayType(StringType())))

df = preprocessed_df.withColumn("Attacker_Helper", str_to_array_of_arrays_udf(preprocessed_df["Attacker_Helper"])).withColumn("Victim", str_to_array_of_arrays_udf(preprocessed_df["Victim"]))

df.printSchema()

root
 |-- Conversation_ID: string (nullable = true)
 |-- Attacker_Helper: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- Victim: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- Conversation_Type: integer (nullable = true)



### Loading the pipeline and transforming the data

In [ ]:
from pyspark.ml import PipelineModel
from src.CustonTransformers import FlattenTransformer

pipeline_model_path = "./Pipelines/TF-IDF_Pipeline"

pipeline = PipelineModel.load(path=pipeline_model_path)

df_assembled = pipeline.transform(df)

### Splitting the Data

In [ ]:
(train_data, test_data) = df_assembled.randomSplit([0.8, 0.2], seed=42)
train_data.show(truncate=False)

+---------------+-----------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## RANDOM FOREST

### Training Random Forest Model

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder



# Define Random Forest Classifier
rf = RandomForestClassifier(labelCol="Conversation_Type", featuresCol="combined_features", predictionCol="Prediction")

evaluator = MulticlassClassificationEvaluator(labelCol='Conversation_Type', metricName="accuracy", predictionCol="Prediction")

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20, 30, 100]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .addGrid(rf.maxBins, [32, 64, 128]) \
    .build()
    
    
# This is similar to cross validation but only split the set once
tvs = TrainValidationSplit(
    estimator=rf, 
    estimatorParamMaps=paramGrid, 
    evaluator=evaluator, 
    parallelism=2, 
    seed=42)

tvsModel = tvs.fit(train_data)

print(tvsModel, '\n')
print(tvsModel.explainParams(), '\n')
print('\nEvaluating with metric -> {}'.format(evaluator.getMetricName()))
print('Evaluators rating for the test set -> {}'.format(evaluator.evaluate(tvsModel.transform(test_data))))


TrainValidationSplitModel_74df925bd2c8 

estimator: estimator to be cross-validated (current: RandomForestClassifier_9d6e0ca72e41)
estimatorParamMaps: estimator param maps (current: [{Param(parent='RandomForestClassifier_9d6e0ca72e41', name='numTrees', doc='Number of trees to train (>= 1).'): 10, Param(parent='RandomForestClassifier_9d6e0ca72e41', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'): 5, Param(parent='RandomForestClassifier_9d6e0ca72e41', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 32}, {Param(parent='RandomForestClassifier_9d6e0ca72e41', name='numTrees', doc='Number of trees to train (>= 1).'): 10, Param(parent='RandomForestClassifier_9d6e0ca72e41', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 int

In [ ]:
print('Train Validation Split Model validationMetrics -> {}'.format(tvsModel.validationMetrics))
print('Best Model -> {}'.format(tvsModel.bestModel))

Train Validation Split Model validationMetrics -> [0.8636363636363636, 0.8636363636363636, 0.8636363636363636, 0.7727272727272727, 0.7727272727272727, 0.7727272727272727, 0.7727272727272727, 0.7727272727272727, 0.7727272727272727, 0.9545454545454546, 0.9545454545454546, 0.9545454545454546, 0.9545454545454546, 0.9545454545454546, 0.9545454545454546, 0.9545454545454546, 0.9545454545454546, 0.9545454545454546, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091, 0.9090909090909091]
Best Model -> RandomForestClassificationModel: uid=RandomForestClassifier_9d6e0ca72e41, numTrees=20, numClasses=2, numFeatures=36


### Testing

In [ ]:
predictions = tvsModel.bestModel.transform(test_data)
predictions.select("Conversation_ID","Conversation_Type", "Prediction", "Probability").where(predictions.Prediction != predictions.Conversation_Type).show(10, truncate=False)

+---------------+-----------------+----------+----------------------------------------+
|Conversation_ID|Conversation_Type|Prediction|Probability                             |
+---------------+-----------------+----------+----------------------------------------+
|4HKpAeRsVhutBr |1                |0.0       |[0.5621659189991786,0.4378340810008215] |
|ALtIzDsxTHTxIe |0                |1.0       |[0.21991697961054948,0.7800830203894505]|
|DjQIAADidtJ3sU |1                |0.0       |[0.5294686840253343,0.4705313159746657] |
|GIjmhRS9Dy8xg1 |0                |1.0       |[0.3750890021738433,0.6249109978261568] |
+---------------+-----------------+----------+----------------------------------------+



In [ ]:
model_path = "../Models/Trained_Models/RandomForest"

In [3]:
tvsModel.write().overwrite().save(model_path)

NameError: name 'tvsModel' is not defined